In [1]:
#Importing parameters
using PyCall
unshift!(PyVector(pyimport("sys")["path"]), "");
@pyimport parameters_julia

out_dir = parameters_julia.out_dir
files_ID = parameters_julia.files_ID
month_w = parameters_julia.month_w
year = parameters_julia.year
instances_ = parameters_julia.instances_ID


instance_ = instances_[3];

In [ ]:
using JSON

instance1 = instance_

#load OD pair-route incidence
odPairRoute = readstring(out_dir * "od_pair_route_incidence_" * instance1 *  files_ID * ".json");
odPairRoute = JSON.parse(odPairRoute);

#load link-route incidence
linkRoute = readstring(out_dir * "link_route_incidence_" * instance1 *  files_ID * ".json");
linkRoute = JSON.parse(linkRoute);

#load OD pair labels
odPairLabel = readstring(out_dir *  "od_pair_label_dict_refined.json");
odPairLabel = JSON.parse(odPairLabel);

odPairLabel_ = readstring(out_dir  *"od_pair_label_dict__refined.json");
odPairLabel_ = JSON.parse(odPairLabel_);

#load link labels
linkLabel = readstring(out_dir * "link_label_dict.json");
linkLabel = JSON.parse(linkLabel);

linkLabel_ = readstring(out_dir * "link_label_dict_.json");
linkLabel_ = JSON.parse(linkLabel_);

#load node-link incidence
nodeLink = readstring(out_dir * "node_link_incidence.json");
nodeLink = JSON.parse(nodeLink);

include("Julia_files/load_network_uni_class.jl");

ta_data = load_ta_network_(out_dir, files_ID, month_w, instance1);

capacity = ta_data.capacity;
free_flow_time = ta_data.free_flow_time;

############
#Read in the demand file
file = open(out_dir * "data_traffic_assignment_uni-class/" * files_ID * "_trips_" * month_w * "_" * instance1 * ".txt")
demands = Dict()
s = 0
for line in eachline(file)
    if contains(line, "Origin")
        s = Int(parse(Float64,(split(line)[2])))
    elseif contains(line, ";")
        pairs = split(line, ";")
        for pair in pairs[1:end-1]
            if !contains(pair, "\n")
                pair_vals = split(pair, ":")
                t, demand = Int(parse(Float64,pair_vals[1])), parse(Float64,pair_vals[2])
                demands[(s,t)] = demand 
            end
        end
    end
end                
close(file);

            